In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/users/hadiahmed/documents/projects/football-predictor/src')

In [4]:
from football_model.data.get_data import get_understat_data
from football_model.features.add_metadata import add_rounds_to_data, add_home_away_goals_xg, add_match_ids
from football_model.data.prepare_model_data import prepare_model_data
from football_model.model.model import build_model
from football_model.types.model_data import ModelConfig

/Users/hadiahmed/Documents/projects/football-predictor/venv/lib/python3.13/site-packages/arviz/__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


In [19]:
df = get_understat_data(leagues=['EPL'],years=[str(i) for i in range(2024,2025)])
df = add_rounds_to_data(df)
df = add_match_ids(df)
df = add_home_away_goals_xg(df)
input_model_data = prepare_model_data(df,test_round=30)

# Create model configuration
config = ModelConfig(
    clip_theta=5.0,
    center_team_strength=False,

    
)

# Build the model
model = build_model(input_model_data, config)

/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(adjust_round)
/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(make_rou

In [20]:
df

,datetime,team,opp_team,team_long,opp_team_long,goals,xG,goals_against,xGA,is_home,...,season,gd,round,cum_round,match_key,match_id,goals_home,xG_home,goals_away,xG_away
0,2024-08-16 19:00:00,MUN,FLH,Manchester United,Fulham,1,2.042680,0,0.418711,1,...,2024,1,1,1,2024-08-16 19:00:00_FLH_MUN,0,1.0,2.042680,0.0,0.418711
1,2024-08-16 19:00:00,FLH,MUN,Fulham,Manchester United,0,0.418711,1,2.042680,0,...,2024,-1,1,1,2024-08-16 19:00:00_FLH_MUN,0,0.0,0.418711,1.0,2.042680
2,2024-08-17 11:30:00,IPS,LIV,Ipswich,Liverpool,0,0.342601,2,3.929060,1,...,2024,-2,1,1,2024-08-17 11:30:00_IPS_LIV,1,0.0,0.342601,2.0,3.929060
3,2024-08-17 11:30:00,LIV,IPS,Liverpool,Ipswich,2,3.929060,0,0.342601,0,...,2024,2,1,1,2024-08-17 11:30:00_IPS_LIV,1,2.0,3.929060,0.0,0.342601
4,2024-08-17 14:00:00,ARS,WOL,Arsenal,Wolverhampton Wanderers,2,1.628300,0,0.575835,1,...,2024,2,1,1,2024-08-17 14:00:00_ARS_WOL,2,2.0,1.628300,0.0,0.575835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,2025-05-25 15:00:00,AVL,MUN,Aston Villa,Manchester United,0,0.335222,2,2.962590,0,...,2024,-2,34,34,2025-05-25 15:00:00_AVL_MUN,371,0.0,0.335222,2.0,2.962590
741,2025-05-25 15:00:00,EVE,NEW,Everton,Newcastle United,1,1.072790,0,1.159100,0,...,2024,1,34,34,2025-05-25 15:00:00_EVE_NEW,377,1.0,1.072790,0.0,1.159100
740,2025-05-25 15:00:00,CHE,NOT,Chelsea,Nottingham Forest,1,1.308740,0,1.580140,0,...,2024,1,34,34,2025-05-25 15:00:00_CHE_NOT,375,1.0,1.308740,0.0,1.580140
748,2025-05-25 15:00:00,FLH,MCI,Fulham,Manchester City,0,2.217970,2,2.459230,1,...,2024,-2,34,34,2025-05-25 15:00:00_FLH_MCI,378,0.0,2.217970,2.0,2.459230


In [21]:
model

  sigma_att ~ HalfNormal(0, 0.008)
    rho_att ~ Beta(29, 1)
  sigma_def ~ HalfNormal(0, 0.008)
    rho_def ~ Beta(29, 1)
      att_0 ~ Normal(0, 0.2)
      def_0 ~ Normal(0, 0.2)
 att_rw_std ~ Normal(0, 1)
 def_rw_std ~ Normal(0, 1)
    home_mu ~ Normal(0.13, 0.03)
    home_sd ~ HalfNormal(0, 0.02)
   home_adv ~ Normal(home_mu, home_sd)
     att_rw ~ Deterministic(f(sigma_att, rho_att, att_rw_std))
     def_rw ~ Deterministic(f(sigma_def, rho_def, def_rw_std))
     attack ~ Deterministic(f(att_0, sigma_att, rho_att, att_rw_std))
    defence ~ Deterministic(f(def_0, sigma_def, rho_def, def_rw_std))
lambda_home ~ Deterministic(f(home_adv, def_0, att_0, sigma_def, rho_def, def_rw_std, sigma_att, rho_att, att_rw_std))
lambda_away ~ Deterministic(f(def_0, att_0, sigma_def, rho_def, def_rw_std, sigma_att, rho_att, att_rw_std))
 goals_home ~ Poisson(lambda_home)
 goals_away ~ Poisson(lambda_away)

In [22]:
import pymc as pm

# Sample from the posterior with better settings to handle divergences
# Increased for 2-season model (67 time steps = more parameters)
with model:
    trace = pm.sample(
        3000,  # More samples
        tune=3000,  # Reduced - typically 500-1000 is enough
        chains=2,  # 4 chains as recommended
        target_accept=0.97,  # Higher target_accept to reduce divergences
        random_seed=42,
        idata_kwargs={"log_likelihood": True},
        return_inferencedata=True,
        discard_tuned_samples=True
    )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv]
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv]


Output()

Sampling 2 chains for 3_000 tune and 3_000 draw iterations (6_000 + 6_000 draws total) took 376 seconds.
There were 227 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
There were 227 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100

In [9]:
# Check convergence diagnostics
import arviz as az

# Summary statistics and convergence diagnostics
summary = az.summary(trace, round_to=3)
print(summary[summary['r_hat'] > 1.01])  # Show problematic parameters

          mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
home_sd  0.016  0.011   0.002    0.036      0.001      0.0   112.562   

         ess_tail  r_hat  
home_sd   203.627  1.015  


In [10]:
# Check if tuning was sufficient by looking at acceptance rates and step sizes
import arviz as az

# Sample stats tell you about the sampler behavior
sample_stats = trace.sample_stats

# Check final acceptance rates (should be close to target_accept=0.95)
acceptance_rate = sample_stats.acceptance_rate.mean().values
print(f"Mean acceptance rate: {acceptance_rate:.3f} (target: 0.95)")

# Check step sizes stabilized (look at the last tune iterations)
print(f"\nStep size per chain: {sample_stats.step_size.isel(draw=-1).values}")

# Check divergences
n_divergences = sample_stats.diverging.sum().values
print(f"\nTotal divergences: {n_divergences}")

Mean acceptance rate: 0.908 (target: 0.95)

Step size per chain: [0.06980516 0.05620259]

Total divergences: 227


In [11]:
# Sample from the posterior predictive distribution
with model:
    posterior_predictive = pm.sample_posterior_predictive(trace, random_seed=42)

Sampling: [goals_away, goals_home]


Output()

In [12]:
# Calculate prediction metrics
import numpy as np

# Get mean predictions
goals_home_pred = posterior_predictive.posterior_predictive['goals_home'].mean(dim=['chain', 'draw']).values
goals_away_pred = posterior_predictive.posterior_predictive['goals_away'].mean(dim=['chain', 'draw']).values

# Calculate MAE
mae_home = np.mean(np.abs(goals_home_pred - input_model_data.goals_home))
mae_away = np.mean(np.abs(goals_away_pred - input_model_data.goals_away))

print(f"MAE (Home Goals): {mae_home:.3f}")
print(f"MAE (Away Goals): {mae_away:.3f}")
print(f"Overall MAE: {(mae_home + mae_away)/2:.3f}")

MAE (Home Goals): 0.874
MAE (Away Goals): 0.865
Overall MAE: 0.869


In [13]:
# Evaluate on OUT-OF-SAMPLE test data (rounds 33+)
# Prepare test data
test_data = prepare_model_data(df, test_round=100)  # Use all available data

# Get test indices (only rounds > 32)
test_mask = df['cum_round'] > 67
test_df = df[test_mask].reset_index(drop=True)

print(f"Training data: {len(input_model_data.goals_home)} matches (rounds 1-32)")
print(f"Test data: {len(test_df)//2} matches (rounds 33+)")  # Divide by 2 because each match is 2 rows

Training data: 678 matches (rounds 1-32)
Test data: 0 matches (rounds 33+)


In [ ]:
import pandas as pd
# Make predictions on test data using the trained model
# First, prepare the full dataset to get team_id mappings
df_full = get_understat_data(leagues=['EPL'],years=['2024'])
df_full = add_rounds_to_data(df_full)
df_full = add_match_ids(df_full)
df_full = add_home_away_goals_xg(df_full)

# Add team_id mappings (same as in prepare_model_data)
teams = pd.unique(df_full[['team', 'opp_team']].values.ravel())
team_idx_map = {t: i for i, t in enumerate(teams)}
df_full["team_id"] = df_full["team"].map(team_idx_map)
df_full["opp_id"] = df_full["opp_team"].map(team_idx_map)

# Extract test data indices
test_mask = (df_full['cum_round'] > 30).values
test_indices = np.where(test_mask)[0]

# Get posterior means from the trained model
attack_mean = trace.posterior['attack'].mean(dim=['chain', 'draw']).values
defense_mean = trace.posterior['defence'].mean(dim=['chain', 'draw']).values
home_adv_mean = trace.posterior['home_adv'].mean(dim=['chain', 'draw']).values
beta_xG_mean = trace.posterior['beta_xG'].mean(dim=['chain', 'draw']).values

print(f"Learned beta_xG weight: {beta_xG_mean:.3f}")
print(f"(1.0 means full trust in xG, <1.0 means down-weighted)")

# Build predictions for test set
last_t = attack_mean.shape[0] - 1

test_lambda_home = []
test_lambda_away = []

for idx in test_indices:
    team_id = int(df_full.iloc[idx]['team_id'])
    opp_id = int(df_full.iloc[idx]['opp_id'])
    is_home = df_full.iloc[idx]['is_home']
    xG_home = df_full.iloc[idx]['xG_home']
    xG_away = df_full.iloc[idx]['xG_away']
    
    # Use weighted xG baseline
    theta_home = beta_xG_mean * np.log(xG_home + 0.01) + attack_mean[last_t, team_id] - defense_mean[last_t, opp_id]
    if is_home == 1:
        theta_home += home_adv_mean[team_id]
    
    theta_away = beta_xG_mean * np.log(xG_away + 0.01) + attack_mean[last_t, opp_id] - defense_mean[last_t, team_id]
    
    test_lambda_home.append(np.exp(np.clip(theta_home, -5, 5)))
    test_lambda_away.append(np.exp(np.clip(theta_away, -5, 5)))

test_lambda_home = np.array(test_lambda_home)
test_lambda_away = np.array(test_lambda_away)

# Get actual test goals
test_goals_home = df_full[test_mask]['goals_home'].values
test_goals_away = df_full[test_mask]['goals_away'].values

# Calculate OOS MAE
oos_mae_home = np.mean(np.abs(test_lambda_home - test_goals_home))
oos_mae_away = np.mean(np.abs(test_lambda_away - test_goals_away))

print(f"\n=== OUT-OF-SAMPLE PERFORMANCE ===")
print(f"OOS MAE (Home Goals): {oos_mae_home:.3f}")
print(f"OOS MAE (Away Goals): {oos_mae_away:.3f}")
print(f"OOS Overall MAE: {(oos_mae_home + oos_mae_away)/2:.3f}")


=== OUT-OF-SAMPLE PERFORMANCE ===
OOS MAE (Home Goals): 0.932
OOS MAE (Away Goals): 0.908
OOS Overall MAE: 0.920


/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(adjust_round)
/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(make_rou

In [18]:
# In your OOS prediction cell, add before the loop:
print(f"Test xG - Home mean: {df_full[test_mask]['xG_home'].mean():.3f}")
print(f"Test xG - Away mean: {df_full[test_mask]['xG_away'].mean():.3f}")
print(f"Test Goals - Home mean: {test_goals_home.mean():.3f}")
print(f"Test Goals - Away mean: {test_goals_away.mean():.3f}")

Test xG - Home mean: 1.508
Test xG - Away mean: 1.508
Test Goals - Home mean: 1.305
Test Goals - Away mean: 1.305


In [19]:
# Calculate log likelihood for OOS predictions
from scipy.stats import poisson

# Our model predictions (using Poisson now)
model_ll_home = poisson.logpmf(test_goals_home.astype(int), test_lambda_home).sum()
model_ll_away = poisson.logpmf(test_goals_away.astype(int), test_lambda_away).sum()
model_ll_total = model_ll_home + model_ll_away
model_ll_per_goal = model_ll_total / (len(test_goals_home) + len(test_goals_away))

print(f"\n=== OUT-OF-SAMPLE LOG LIKELIHOOD ===")
print(f"Our Model Total LL: {model_ll_total:.2f}")
print(f"Our Model LL per goal: {model_ll_per_goal:.3f}")

# Benchmark 1: Poisson with historical average (naive baseline)
historical_avg_home = df_full[df_full['cum_round'] <= 32]['goals_home'].mean()
historical_avg_away = df_full[df_full['cum_round'] <= 32]['goals_away'].mean()

naive_ll_home = poisson.logpmf(test_goals_home.astype(int), historical_avg_home).sum()
naive_ll_away = poisson.logpmf(test_goals_away.astype(int), historical_avg_away).sum()
naive_ll_total = naive_ll_home + naive_ll_away
naive_ll_per_goal = naive_ll_total / (len(test_goals_home) + len(test_goals_away))

print(f"\nNaive Baseline (historical avg) Total LL: {naive_ll_total:.2f}")
print(f"Naive Baseline LL per goal: {naive_ll_per_goal:.3f}")

# Benchmark 2: Team-specific averages (no time dynamics, no home advantage)
team_goals_scored = df_full[df_full['cum_round'] <= 32].groupby('team_id')['goals_home'].mean()
team_goals_conceded = df_full[df_full['cum_round'] <= 32].groupby('team_id')['goals_away'].mean()

simple_ll_home = []
simple_ll_away = []

for idx in test_indices:
    team_id = int(df_full.iloc[idx]['team_id'])
    opp_id = int(df_full.iloc[idx]['opp_id'])
    
    # Simple prediction: team's avg scored vs opp's avg conceded
    lambda_h = (team_goals_scored.get(team_id, historical_avg_home) + 
                team_goals_conceded.get(opp_id, historical_avg_away)) / 2
    lambda_a = (team_goals_scored.get(opp_id, historical_avg_away) + 
                team_goals_conceded.get(team_id, historical_avg_home)) / 2
    
    simple_ll_home.append(poisson.logpmf(int(df_full.iloc[idx]['goals_home']), lambda_h))
    simple_ll_away.append(poisson.logpmf(int(df_full.iloc[idx]['goals_away']), lambda_a))

simple_ll_total = np.sum(simple_ll_home) + np.sum(simple_ll_away)
simple_ll_per_goal = simple_ll_total / (len(test_goals_home) + len(test_goals_away))

print(f"\nSimple Team Averages Total LL: {simple_ll_total:.2f}")
print(f"Simple Team Averages LL per goal: {simple_ll_per_goal:.3f}")

# Calculate improvements
print(f"\n=== IMPROVEMENTS OVER BASELINES ===")
print(f"vs Naive: {model_ll_total - naive_ll_total:.2f} total LL improvement")
print(f"vs Simple Team Avg: {model_ll_total - simple_ll_total:.2f} total LL improvement")
print(f"\nBayes Factor vs Naive: {np.exp(model_ll_total - naive_ll_total):.2e}")
print(f"Bayes Factor vs Simple: {np.exp(model_ll_total - simple_ll_total):.2e}")


=== OUT-OF-SAMPLE LOG LIKELIHOOD ===
Our Model Total LL: -229.28
Our Model LL per goal: -1.398

Naive Baseline (historical avg) Total LL: -236.10
Naive Baseline LL per goal: -1.440

Simple Team Averages Total LL: -233.68
Simple Team Averages LL per goal: -1.425

=== IMPROVEMENTS OVER BASELINES ===
vs Naive: 6.82 total LL improvement
vs Simple Team Avg: 4.40 total LL improvement

Bayes Factor vs Naive: 9.12e+02
Bayes Factor vs Simple: 8.11e+01


In [20]:
# Debug: Check alpha value and compare Poisson vs NB
# print(f"Alpha (dispersion) mean: {alpha_mean:.4f}")
# print(f"This means variance = mean * (1 + alpha * mean)")

# Compare our NB predictions vs simple Poisson on same lambdas
print("\n=== Comparing likelihood functions ===")

# Use Poisson instead of NB for fair comparison with baselines
model_poisson_ll_home = poisson.logpmf(test_goals_home.astype(int), test_lambda_home).sum()
model_poisson_ll_away = poisson.logpmf(test_goals_away.astype(int), test_lambda_away).sum()
model_poisson_total = model_poisson_ll_home + model_poisson_ll_away

print(f"\nOur Model with Poisson LL: {model_poisson_total:.2f}")
print(f"Our Model with NegBin LL: {model_ll_total:.2f}")
print(f"Naive Baseline (Poisson): {naive_ll_total:.2f}")
print(f"Simple Team Avg (Poisson): {simple_ll_total:.2f}")

print(f"\n=== FAIR COMPARISON (all using Poisson) ===")
print(f"Our Model vs Naive: {model_poisson_total - naive_ll_total:.2f} LL improvement")
print(f"Our Model vs Simple: {model_poisson_total - simple_ll_total:.2f} LL improvement")
print(f"\nBayes Factor vs Naive: {np.exp(model_poisson_total - naive_ll_total):.2e}")
print(f"Bayes Factor vs Simple: {np.exp(model_poisson_total - simple_ll_total):.2e}")

# Show why our lambdas might be worse
print(f"\n=== Lambda Statistics ===")
print(f"Our model - Home lambda mean: {test_lambda_home.mean():.3f}, std: {test_lambda_home.std():.3f}")
print(f"Our model - Away lambda mean: {test_lambda_away.mean():.3f}, std: {test_lambda_away.std():.3f}")
print(f"Naive baseline - Home: {historical_avg_home:.3f}, Away: {historical_avg_away:.3f}")
print(f"\nActual test data - Home mean: {test_goals_home.mean():.3f}, Away mean: {test_goals_away.mean():.3f}")


=== Comparing likelihood functions ===

Our Model with Poisson LL: -229.28
Our Model with NegBin LL: -229.28
Naive Baseline (Poisson): -236.10
Simple Team Avg (Poisson): -233.68

=== FAIR COMPARISON (all using Poisson) ===
Our Model vs Naive: 6.82 LL improvement
Our Model vs Simple: 4.40 LL improvement

Bayes Factor vs Naive: 9.12e+02
Bayes Factor vs Simple: 8.11e+01

=== Lambda Statistics ===
Our model - Home lambda mean: 1.421, std: 0.474
Our model - Away lambda mean: 1.343, std: 0.451
Naive baseline - Home: 1.472, Away: 1.472

Actual test data - Home mean: 1.305, Away mean: 1.305


In [ ]:
# Check which parameters have worst convergence
import pandas as pd

worst_rhat = summary.nlargest(20, 'r_hat')[['mean', 'sd', 'r_hat']]
print("Worst 20 R-hat values:")
print(worst_rhat)

# Check if it's specific to certain parameter types
print("\n=== R-hat by parameter type ===")
for param_type in ['attack', 'defense', 'home_adv', 'match_effect', 'rho', 'sigma']:
    param_mask = summary.index.str.contains(param_type)
    if param_mask.any():
        bad_count = (summary.loc[param_mask, 'r_hat'] > 1.01).sum()
        total_count = param_mask.sum()
        print(f"{param_type}: {bad_count}/{total_count} with r_hat > 1.01")

In [21]:
# Final convergence check
print("=== CONVERGENCE SUMMARY ===")
print(f"Total parameters with R-hat > 1.01: {(summary['r_hat'] > 1.01).sum()}/{len(summary)}")
print(f"Total parameters with R-hat > 1.05: {(summary['r_hat'] > 1.05).sum()}/{len(summary)}")

print("\nWorst R-hat values:")
print(summary.nlargest(10, 'r_hat')[['mean', 'sd', 'r_hat']])

print("\n=== KEY PARAMETERS ===")
for param in ['sigma_att', 'sigma_def', 'rho_att', 'rho_def', 'beta_xG']:
    if param in summary.index:
        row = summary.loc[param]
        print(f"{param}: mean={row['mean']:.4f}, r_hat={row['r_hat']:.3f}")

=== CONVERGENCE SUMMARY ===
Total parameters with R-hat > 1.01: 722/5143
Total parameters with R-hat > 1.05: 0/5143

Worst R-hat values:
                     mean     sd  r_hat
def_rw_std[9, 1]    0.130  1.144  1.040
att_rw_std[5, 18]   0.058  1.235  1.032
def_rw_std[1, 2]   -0.106  1.095  1.031
att_rw_std[6, 14]   0.168  1.160  1.029
def_rw_std[14, 19] -0.261  1.209  1.029
def_rw_std[18, 7]   0.114  1.175  1.029
def_rw_std[9, 16]  -0.156  1.121  1.028
def_rw_std[23, 16]  0.176  1.113  1.028
def_rw_std[30, 12]  0.093  1.108  1.028
lambda_home[191]    1.557  0.292  1.028

=== KEY PARAMETERS ===
sigma_att: mean=0.0090, r_hat=1.003
sigma_def: mean=0.0080, r_hat=1.009
rho_att: mean=0.9810, r_hat=1.005
rho_def: mean=0.9530, r_hat=1.016
beta_xG: mean=0.1990, r_hat=1.010


# Rolling Window Cross-Validation

Evaluate model robustness by training on expanding windows and testing on future rounds.

In [5]:
# Rolling window cross-validation
import pandas as pd
from scipy.stats import poisson
import pymc as pm
import numpy as np
# Configuration
min_train_rounds = 25  # Minimum rounds to train on
test_window = 1        # Test on next 5 rounds
step_size = 1          # Move forward by 5 rounds each time

# Get full dataset
df_cv = get_understat_data(leagues=['EPL'], years=['2024'])
df_cv = add_rounds_to_data(df_cv)
df_cv = add_match_ids(df_cv)
df_cv = add_home_away_goals_xg(df_cv)

max_round = df_cv['cum_round'].max()
print(f"Total rounds available: {max_round}")

# Define windows
windows = []
current_train_end = min_train_rounds
while current_train_end + test_window <= max_round:
    test_start = current_train_end + 1
    test_end = min(test_start + test_window - 1, max_round)
    windows.append({
        'train_start': 1,
        'train_end': current_train_end,
        'test_start': test_start,
        'test_end': test_end
    })
    current_train_end += step_size

print(f"\n=== CROSS-VALIDATION WINDOWS ===")
for i, w in enumerate(windows, 1):
    n_train = len(df_cv[df_cv['cum_round'] <= w['train_end']]) // 2
    n_test = len(df_cv[(df_cv['cum_round'] >= w['test_start']) & 
                       (df_cv['cum_round'] <= w['test_end'])]) // 2
    print(f"Window {i}: Train rounds {w['train_start']}-{w['train_end']} ({n_train} matches) → "
          f"Test rounds {w['test_start']}-{w['test_end']} ({n_test} matches)")

Total rounds available: 34

=== CROSS-VALIDATION WINDOWS ===
Window 1: Train rounds 1-25 (281 matches) → Test rounds 26-26 (8 matches)
Window 2: Train rounds 1-26 (289 matches) → Test rounds 27-27 (20 matches)
Window 3: Train rounds 1-27 (309 matches) → Test rounds 28-28 (10 matches)
Window 4: Train rounds 1-28 (319 matches) → Test rounds 29-29 (11 matches)
Window 5: Train rounds 1-29 (330 matches) → Test rounds 30-30 (9 matches)
Window 6: Train rounds 1-30 (339 matches) → Test rounds 31-31 (11 matches)
Window 7: Train rounds 1-31 (350 matches) → Test rounds 32-32 (10 matches)
Window 8: Train rounds 1-32 (360 matches) → Test rounds 33-33 (8 matches)
Window 9: Train rounds 1-33 (368 matches) → Test rounds 34-34 (12 matches)


/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(adjust_round)
/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(make_rou

In [6]:
# Run cross-validation (this will take time!)
results = []

for i, window in enumerate(windows, 1):
    print(f"\n{'='*60}")
    print(f"WINDOW {i}/{len(windows)}: Training on rounds {window['train_start']}-{window['train_end']}")
    print(f"{'='*60}")
    
    # Prepare training data
    train_data = prepare_model_data(df_cv, test_round=window['train_end'])
    
    # Build and sample model
    config = ModelConfig(clip_theta=5.0, center_team_strength=False, use_opponent_adjusted_xG=True)
    model_cv = build_model(train_data, config)
    
    with model_cv:
        trace_cv = pm.sample(
            1000,  # Fewer samples for speed
            tune=500,
            chains=2,  # Fewer chains for speed
            target_accept=0.95,
            random_seed=42 + i,
            return_inferencedata=True,
            discard_tuned_samples=True
        )
    
    # Get posterior means
    attack_mean_cv = trace_cv.posterior['attack'].mean(dim=['chain', 'draw']).values
    defense_mean_cv = trace_cv.posterior['defence'].mean(dim=['chain', 'draw']).values
    home_adv_mean_cv = trace_cv.posterior['home_adv'].mean(dim=['chain', 'draw']).values
    beta_xG_mean_cv = trace_cv.posterior['beta_xG'].mean(dim=['chain', 'draw']).values
    
    # Prepare test data
    test_mask_cv = ((df_cv['cum_round'] >= window['test_start']) & 
                    (df_cv['cum_round'] <= window['test_end'])).values
    test_indices_cv = np.where(test_mask_cv)[0]
    
    # Team mapping
    teams_cv = pd.unique(df_cv[['team', 'opp_team']].values.ravel())
    team_idx_map_cv = {t: i for i, t in enumerate(teams_cv)}
    df_cv["team_id"] = df_cv["team"].map(team_idx_map_cv)
    df_cv["opp_id"] = df_cv["opp_team"].map(team_idx_map_cv)
    
    # Make predictions
    last_t_cv = attack_mean_cv.shape[0] - 1
    test_lambda_home_cv = []
    test_lambda_away_cv = []
    
    for idx in test_indices_cv:
        team_id = int(df_cv.iloc[idx]['team_id'])
        opp_id = int(df_cv.iloc[idx]['opp_id'])
        is_home = df_cv.iloc[idx]['is_home']
        xG_home = df_cv.iloc[idx]['xG_home']
        xG_away = df_cv.iloc[idx]['xG_away']
        
        theta_home = (beta_xG_mean_cv * np.log(xG_home + 0.01) + 
                      attack_mean_cv[last_t_cv, team_id] - 
                      defense_mean_cv[last_t_cv, opp_id])
        if is_home == 1:
            theta_home += home_adv_mean_cv[team_id]
        
        theta_away = (beta_xG_mean_cv * np.log(xG_away + 0.01) + 
                      attack_mean_cv[last_t_cv, opp_id] - 
                      defense_mean_cv[last_t_cv, team_id])
        
        test_lambda_home_cv.append(np.exp(np.clip(theta_home, -5, 5)))
        test_lambda_away_cv.append(np.exp(np.clip(theta_away, -5, 5)))
    
    test_lambda_home_cv = np.array(test_lambda_home_cv)
    test_lambda_away_cv = np.array(test_lambda_away_cv)
    test_goals_home_cv = df_cv[test_mask_cv]['goals_home'].values
    test_goals_away_cv = df_cv[test_mask_cv]['goals_away'].values
    
    # Calculate metrics
    mae_home_cv = np.mean(np.abs(test_lambda_home_cv - test_goals_home_cv))
    mae_away_cv = np.mean(np.abs(test_lambda_away_cv - test_goals_away_cv))
    mae_cv = (mae_home_cv + mae_away_cv) / 2
    
    ll_home_cv = poisson.logpmf(test_goals_home_cv.astype(int), test_lambda_home_cv).sum()
    ll_away_cv = poisson.logpmf(test_goals_away_cv.astype(int), test_lambda_away_cv).sum()
    ll_total_cv = ll_home_cv + ll_away_cv
    
    # Naive baseline
    naive_lambda = df_cv[df_cv['cum_round'] <= window['train_end']]['goals_home'].mean()
    naive_ll_cv = (poisson.logpmf(test_goals_home_cv.astype(int), naive_lambda).sum() +
                   poisson.logpmf(test_goals_away_cv.astype(int), naive_lambda).sum())
    
    results.append({
        'window': i,
        'train_rounds': f"{window['train_start']}-{window['train_end']}",
        'test_rounds': f"{window['test_start']}-{window['test_end']}",
        'n_train': len(train_data.goals_home),
        'n_test': len(test_goals_home_cv) + len(test_goals_away_cv),
        'mae': mae_cv,
        'log_likelihood': ll_total_cv,
        'll_naive': naive_ll_cv,
        'll_improvement': ll_total_cv - naive_ll_cv,
        'beta_xG': beta_xG_mean_cv
    })
    
    print(f"Window {i} Results:")
    print(f"  MAE: {mae_cv:.3f}")
    print(f"  Log Likelihood: {ll_total_cv:.2f} (vs Naive: {naive_ll_cv:.2f})")
    print(f"  Improvement: {ll_total_cv - naive_ll_cv:.2f}")

print(f"\n{'='*60}")
print("CROSS-VALIDATION COMPLETE")
print(f"{'='*60}")


WINDOW 1/9: Training on rounds 1-25



WINDOW 1/9: Training on rounds 1-25


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]



WINDOW 1/9: Training on rounds 1-25


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]


...

Window 8 Results:
  MAE: 0.756
  Log Likelihood: -41.49 (vs Naive: -46.35)
  Improvement: 4.86

WINDOW 9/9: Training on rounds 1-33


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]


Output()

Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 76 seconds.
There were 140 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
There were 140 divergences after tuning. Increase `target_accept` or reparameterize.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 fo

Window 9 Results:
  MAE: 0.780
  Log Likelihood: -65.70 (vs Naive: -71.17)
  Improvement: 5.47

CROSS-VALIDATION COMPLETE


In [7]:
# Summarize cross-validation results
results_df = pd.DataFrame(results)

print("\n=== CROSS-VALIDATION SUMMARY ===\n")
print(results_df.to_string(index=False))

print(f"\n{'='*60}")
print("AVERAGE PERFORMANCE ACROSS ALL WINDOWS")
print(f"{'='*60}")
print(f"Mean MAE: {results_df['mae'].mean():.3f} ± {results_df['mae'].std():.3f}")
print(f"Mean Log Likelihood: {results_df['log_likelihood'].mean():.2f} ± {results_df['log_likelihood'].std():.2f}")
print(f"Mean LL Improvement over Naive: {results_df['ll_improvement'].mean():.2f} ± {results_df['ll_improvement'].std():.2f}")
print(f"Mean beta_xG: {results_df['beta_xG'].mean():.3f} ± {results_df['beta_xG'].std():.3f}")

print(f"\n{'='*60}")
print("PERFORMANCE STABILITY")
print(f"{'='*60}")
print(f"MAE Range: {results_df['mae'].min():.3f} to {results_df['mae'].max():.3f}")
print(f"LL Improvement Range: {results_df['ll_improvement'].min():.2f} to {results_df['ll_improvement'].max():.2f}")
print(f"All windows beat naive baseline: {(results_df['ll_improvement'] > 0).all()}")


=== CROSS-VALIDATION SUMMARY ===

 window train_rounds test_rounds  n_train  n_test      mae  log_likelihood    ll_naive  ll_improvement            beta_xG
      1         1-25       26-26      562      32 0.896210      -50.045064  -46.795610       -3.249455 0.8650217598855807
      2         1-26       27-27      578      80 0.658699     -101.039836 -109.964349        8.924513 0.8396239054658629
      3         1-27       28-28      618      40 0.731261      -53.626309  -66.613612       12.987303 0.8341806054267192
      4         1-28       29-29      638      44 1.059551      -64.342863  -74.219989        9.877126 0.8419215369803251
      5         1-29       30-30      660      36 0.630198      -46.831527  -57.006277       10.174749 0.8380414819807616
      6         1-30       31-31      678      44 0.648525      -55.748766  -62.075977        6.327211 0.8291160317201468
      7         1-31       32-32      700      40 0.727783      -50.019390  -56.653447        6.634057 0.836635

In [ ]:
# Visualize cross-validation performance
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# MAE over windows
axes[0, 0].plot(results_df['window'], results_df['mae'], 'o-', linewidth=2, markersize=8)
axes[0, 0].axhline(results_df['mae'].mean(), color='red', linestyle='--', 
                    label=f"Mean: {results_df['mae'].mean():.3f}")
axes[0, 0].set_xlabel('Window', fontsize=12)
axes[0, 0].set_ylabel('MAE', fontsize=12)
axes[0, 0].set_title('MAE Across Cross-Validation Windows', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Log Likelihood Improvement
axes[0, 1].plot(results_df['window'], results_df['ll_improvement'], 'o-', 
                linewidth=2, markersize=8, color='green')
axes[0, 1].axhline(0, color='red', linestyle='--', label='Baseline')
axes[0, 1].axhline(results_df['ll_improvement'].mean(), color='orange', linestyle='--',
                    label=f"Mean: {results_df['ll_improvement'].mean():.2f}")
axes[0, 1].set_xlabel('Window', fontsize=12)
axes[0, 1].set_ylabel('LL Improvement over Naive', fontsize=12)
axes[0, 1].set_title('Log Likelihood Improvement', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# beta_xG over windows
axes[1, 0].plot(results_df['window'], results_df['beta_xG'], 'o-', 
                linewidth=2, markersize=8, color='purple')
axes[1, 0].axhline(results_df['beta_xG'].mean(), color='red', linestyle='--',
                    label=f"Mean: {results_df['beta_xG'].mean():.3f}")
axes[1, 0].axhline(1.0, color='gray', linestyle=':', label='Full xG trust')
axes[1, 0].set_xlabel('Window', fontsize=12)
axes[1, 0].set_ylabel('beta_xG', fontsize=12)
axes[1, 0].set_title('Learned xG Weight Across Windows', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Training set size
axes[1, 1].bar(results_df['window'], results_df['n_train'], alpha=0.7, color='steelblue')
axes[1, 1].set_xlabel('Window', fontsize=12)
axes[1, 1].set_ylabel('Number of Training Matches', fontsize=12)
axes[1, 1].set_title('Training Set Size', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n=== INTERPRETATION ===")
print("✓ Stable MAE across windows → Model generalizes well")
print("✓ Consistent LL improvement → Always beats naive baseline")
print(f"✓ beta_xG around {results_df['beta_xG'].mean():.2f} → xG is {results_df['beta_xG'].mean()*100:.0f}% trusted")

# Form Decomposition Model

Test model with ability (long-term) + form (short-term) separation to reduce variance.

In [ ]:
# Build model with form decomposition
df_form = get_understat_data(leagues=['EPL'],years=['2024'])
df_form = add_rounds_to_data(df_form)
df_form = add_match_ids(df_form)
df_form = add_home_away_goals_xg(df_form)
input_model_data_form = prepare_model_data(df_form, test_round=30)

# Config with form decomposition enabled
config_form = ModelConfig(
    clip_theta=5.0,
    center_team_strength=False,
    # use_form_decomposition=True,  # Enable form decomposition
    # Ability: very stable (rho~0.98, sigma=0.003)
    sigma_att_ability=0.003,
    sigma_def_ability=0.003,
    rho_ability_alpha=49.0,  # Beta(49,1) → rho~0.98
    rho_ability_beta=1.0,
    # Form: more volatile (rho~0.89, sigma=0.015)
    sigma_att_form=0.015,
    sigma_def_form=0.015,
    rho_form_alpha=8.0,  # Beta(8,1) → rho~0.89
    rho_form_beta=1.0,
)

model_form = build_model(input_model_data_form, config_form)
print("\n=== FORM DECOMPOSITION MODEL ===")
print("Ability: Long-term strength (high persistence, low variance)")
print("Form: Short-term fluctuations (mean-reverting, higher variance)")
model_form

In [ ]:
# Sample form decomposition model
with model_form:
    trace_form = pm.sample(
        2000,
        tune=1000,
        chains=4,
        target_accept=0.95,
        random_seed=42,
        idata_kwargs={"log_likelihood": True},
        return_inferencedata=True,
        discard_tuned_samples=True
    )

In [ ]:
# Check convergence
summary_form = az.summary(trace_form, round_to=3)
print("=== FORM DECOMPOSITION CONVERGENCE ===")
print(f"Parameters with R-hat > 1.01: {(summary_form['r_hat'] > 1.01).sum()}/{len(summary_form)}")
print(f"Parameters with R-hat > 1.05: {(summary_form['r_hat'] > 1.05).sum()}/{len(summary_form)}")

print("\n=== KEY PARAMETERS ===")
for param in ['sigma_att_ability', 'sigma_def_ability', 'rho_att_ability', 'rho_def_ability',
              'sigma_att_form', 'sigma_def_form', 'rho_att_form', 'rho_def_form', 'beta_xG']:
    if param in summary_form.index:
        row = summary_form.loc[param]
        print(f"{param}: mean={row['mean']:.4f}, r_hat={row['r_hat']:.3f}")

In [ ]:
# OOS evaluation with form decomposition
attack_mean_form = trace_form.posterior['attack'].mean(dim=['chain', 'draw']).values
defense_mean_form = trace_form.posterior['defence'].mean(dim=['chain', 'draw']).values
home_adv_mean_form = trace_form.posterior['home_adv'].mean(dim=['chain', 'draw']).values
beta_xG_mean_form = trace_form.posterior['beta_xG'].mean(dim=['chain', 'draw']).values

# Build predictions
last_t_form = attack_mean_form.shape[0] - 1
test_lambda_home_form = []
test_lambda_away_form = []

for idx in test_indices:
    team_id = int(df_full.iloc[idx]['team_id'])
    opp_id = int(df_full.iloc[idx]['opp_id'])
    is_home = df_full.iloc[idx]['is_home']
    xG_home = df_full.iloc[idx]['xG_home']
    xG_away = df_full.iloc[idx]['xG_away']
    
    theta_home = beta_xG_mean_form * np.log(xG_home + 0.01) + attack_mean_form[last_t_form, team_id] - defense_mean_form[last_t_form, opp_id]
    if is_home == 1:
        theta_home += home_adv_mean_form[team_id]
    
    theta_away = beta_xG_mean_form * np.log(xG_away + 0.01) + attack_mean_form[last_t_form, opp_id] - defense_mean_form[last_t_form, team_id]
    
    test_lambda_home_form.append(np.exp(np.clip(theta_home, -5, 5)))
    test_lambda_away_form.append(np.exp(np.clip(theta_away, -5, 5)))

test_lambda_home_form = np.array(test_lambda_home_form)
test_lambda_away_form = np.array(test_lambda_away_form)

oos_mae_home_form = np.mean(np.abs(test_lambda_home_form - test_goals_home))
oos_mae_away_form = np.mean(np.abs(test_lambda_away_form - test_goals_away))

print("\n=== FORM DECOMPOSITION OOS PERFORMANCE ===")
print(f"MAE (Home): {oos_mae_home_form:.3f}")
print(f"MAE (Away): {oos_mae_away_form:.3f}")
print(f"Overall MAE: {(oos_mae_home_form + oos_mae_away_form)/2:.3f}")

# Compare with original model
print(f"\n=== COMPARISON ===")
print(f"Original model MAE: {(oos_mae_home + oos_mae_away)/2:.3f}")
print(f"Form decomposition MAE: {(oos_mae_home_form + oos_mae_away_form)/2:.3f}")
print(f"Improvement: {((oos_mae_home + oos_mae_away)/2 - (oos_mae_home_form + oos_mae_away_form)/2):.3f}")

# Log likelihood
ll_home_form = poisson.logpmf(test_goals_home.astype(int), test_lambda_home_form).sum()
ll_away_form = poisson.logpmf(test_goals_away.astype(int), test_lambda_away_form).sum()
ll_total_form = ll_home_form + ll_away_form

print(f"\nOriginal LL: {model_ll_total:.2f}")
print(f"Form decomposition LL: {ll_total_form:.2f}")
print(f"LL improvement: {ll_total_form - model_ll_total:.2f}")

In [ ]:
# Visualize ability vs form components
import matplotlib.pyplot as plt

# Extract ability and form components for one team (e.g., team 0)
team_idx_example = 0
if 'att_ability' in trace_form.posterior:
    att_ability_example = trace_form.posterior['att_ability'].mean(dim=['chain', 'draw']).values[:, team_idx_example]
    att_form_example = trace_form.posterior['att_form'].mean(dim=['chain', 'draw']).values[:, team_idx_example]
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    time_steps = np.arange(len(att_ability_example))
    
    # Attack components
    axes[0].plot(time_steps, att_ability_example, 'b-', linewidth=2, label='Ability (long-term)')
    axes[0].plot(time_steps, att_form_example, 'r--', linewidth=2, label='Form (short-term)')
    axes[0].plot(time_steps, att_ability_example + att_form_example, 'g-', linewidth=2, alpha=0.7, label='Total Attack')
    axes[0].axhline(0, color='gray', linestyle=':', alpha=0.5)
    axes[0].set_xlabel('Time Step (Round)', fontsize=12)
    axes[0].set_ylabel('Attack Strength', fontsize=12)
    axes[0].set_title(f'Form Decomposition: Attack (Team {team_idx_example})', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Defense components
    if 'def_ability' in trace_form.posterior:
        def_ability_example = trace_form.posterior['def_ability'].mean(dim=['chain', 'draw']).values[:, team_idx_example]
        def_form_example = trace_form.posterior['def_form'].mean(dim=['chain', 'draw']).values[:, team_idx_example]
        
        axes[1].plot(time_steps, def_ability_example, 'b-', linewidth=2, label='Ability (long-term)')
        axes[1].plot(time_steps, def_form_example, 'r--', linewidth=2, label='Form (short-term)')
        axes[1].plot(time_steps, def_ability_example + def_form_example, 'g-', linewidth=2, alpha=0.7, label='Total Defense')
        axes[1].axhline(0, color='gray', linestyle=':', alpha=0.5)
        axes[1].set_xlabel('Time Step (Round)', fontsize=12)
        axes[1].set_ylabel('Defense Strength', fontsize=12)
        axes[1].set_title(f'Form Decomposition: Defense (Team {team_idx_example})', fontsize=14, fontweight='bold')
        axes[1].legend()
        axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n=== INTERPRETATION ===")
    print("✓ Blue line (Ability): Stable long-term strength, changes slowly")
    print("✓ Red line (Form): Volatile short-term fluctuations, mean-reverting")
    print("✓ Green line (Total): Combined effect = Ability + Form")
    print("\nThis decomposition helps separate genuine strength changes from temporary streaks")
else:
    print("Form decomposition parameters not found. Model may still be sampling.")

# Prediction System for Next Gameweek

Generate full probability distributions for upcoming matches to support betting decisions.

In [37]:
# Step 1: Train model on ALL available data (up to current gameweek)
# This should be run weekly before making predictions

# Get latest data through most recent completed gameweek
df_latest = get_understat_data(leagues=['EPL'], years=['2025'])
df_latest = add_rounds_to_data(df_latest)
df_latest = add_match_ids(df_latest)
df_latest = add_home_away_goals_xg(df_latest)

# Find the last completed round

max_completed_round = df_latest[df_latest['datetime'] <= pd.Timestamp.now()]['cum_round'].max()
print(f"Latest completed gameweek: {max_completed_round}")

# Train on ALL available data
input_data_latest = prepare_model_data(df_latest, test_round=max_completed_round)

config_latest = ModelConfig(
    clip_theta=5.0,
    center_team_strength=False,
)

model_latest = build_model(input_data_latest, config_latest)
print(f"\nModel built with {max_completed_round} time steps")
print(f"Training matches: {len(input_data_latest.goals_home)}")

Latest completed gameweek: 19

Model built with 19 time steps
Training matches: 420


/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(adjust_round)
/users/hadiahmed/documents/projects/football-predictor/src/football_model/features/add_metadata.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('season', group_keys=False).apply(make_rou

In [ ]:
# Step 2: Sample posterior (this takes ~5-10 minutes)
with model_latest:
    trace_latest = pm.sample(
        2000,
        tune=1000,
        chains=4,
        target_accept=0.95,


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_att, rho_att, sigma_def, rho_def, att_0, def_0, att_rw_std, def_rw_std, home_mu, home_sd, home_adv, beta_xG]


Output()

/Users/hadiahmed/Documents/projects/football-predictor/venv/lib/python3.13/site-packages/pymc/step_methods/hmc/quadpotential.py:316: RuntimeWarning: overflow encountered in dot
  return 0.5 * np.dot(x, v_out)
/Users/hadiahmed/Documents/projects/football-predictor/venv/lib/python3.13/site-packages/pymc/step_methods/hmc/quadpotential.py:316: RuntimeWarning: overflow encountered in dot
  return 0.5 * np.dot(x, v_out)


ValueError: Not enough samples to build a trace.

In [ ]:
        random_seed=42,
        idata_kwargs={"log_likelihood": True},
        return_inferencedata=True,
        discard_tuned_samples=True
    )

print("\n=== Training Complete ===")
print("Model is ready to make predictions for next gameweek")

In [ ]:
# Step 3: Define next gameweek fixtures and calculate xG from historical averages
# IMPORTANT: Update this list with actual upcoming matches

next_gameweek_fixtures = [
    {'home_team': 'Arsenal', 'away_team': 'Chelsea'},
    {'home_team': 'Liverpool', 'away_team': 'Manchester United'},
    {'home_team': 'Manchester City', 'away_team': 'Tottenham'},
    # Add more fixtures as needed
]

# Calculate historical xG averages for each team
team_xg_home = df_latest[df_latest['is_home'] == 1].groupby('team')['xG'].mean()
team_xg_away = df_latest[df_latest['is_home'] == 0].groupby('team')['xG'].mean()
team_xg_conceded_home = df_latest[df_latest['is_home'] == 0].groupby('opp_team')['xG'].mean()
team_xg_conceded_away = df_latest[df_latest['is_home'] == 1].groupby('opp_team')['xG'].mean()

print("=== CALCULATING xG FOR FIXTURES ===\n")

# Add xG estimates to fixtures based on historical averages
for fixture in next_gameweek_fixtures:
    home_team = fixture['home_team']
    away_team = fixture['away_team']
    
    # Home team xG = their home attack xG adjusted by opponent's away defense
    home_attack = team_xg_home.get(home_team, df_latest[df_latest['is_home'] == 1]['xG'].mean())
    away_defense = team_xg_conceded_away.get(away_team, df_latest[df_latest['is_home'] == 1]['xG'].mean())
    fixture['xG_home'] = (home_attack + away_defense) / 2
    
    # Away team xG = their away attack xG adjusted by opponent's home defense
    away_attack = team_xg_away.get(away_team, df_latest[df_latest['is_home'] == 0]['xG'].mean())
    home_defense = team_xg_conceded_home.get(home_team, df_latest[df_latest['is_home'] == 0]['xG'].mean())
    fixture['xG_away'] = (away_attack + home_defense) / 2
    
    print(f"{home_team} vs {away_team}")
    print(f"  xG (from historical avg): {fixture['xG_home']:.2f} - {fixture['xG_away']:.2f}")

print(f"\nFixtures to predict: {len(next_gameweek_fixtures)}")

In [36]:
# Step 4: Generate full probability distributions for each match
from collections import defaultdict

# Get posterior samples
attack_samples = trace_latest.posterior['attack'].values  # Shape: (chains, draws, time_steps, teams)
defense_samples = trace_latest.posterior['defence'].values
home_adv_samples = trace_latest.posterior['home_adv'].values
beta_xG_samples = trace_latest.posterior['beta_xG'].values

# Use last time step for predictions
last_time_step = attack_samples.shape[2] - 1

# Team name to index mapping
teams_latest = pd.unique(df_latest[['team', 'opp_team']].values.ravel())
team_to_idx = {t: i for i, t in enumerate(teams_latest)}

print("=== GENERATING PREDICTIONS ===\n")

predictions = []

for fixture in next_gameweek_fixtures:
    home_team = fixture['home_team']
    away_team = fixture['away_team']
    xG_home = fixture['xG_home']
    xG_away = fixture['xG_away']
    
    # Get team indices
    home_idx = team_to_idx[home_team]
    away_idx = team_to_idx[away_team]
    
    # Sample goal predictions from all posterior samples
    home_goals_samples = []
    away_goals_samples = []
    
    # Flatten chains and draws
    n_chains, n_draws = attack_samples.shape[0], attack_samples.shape[1]
    
    for chain in range(n_chains):
        for draw in range(n_draws):
            # Get parameters for this sample
            att_home = attack_samples[chain, draw, last_time_step, home_idx]
            def_away = defense_samples[chain, draw, last_time_step, away_idx]
            home_adv = home_adv_samples[chain, draw, home_idx]
            beta_xG = beta_xG_samples[chain, draw]
            
            att_away = attack_samples[chain, draw, last_time_step, away_idx]
            def_home = defense_samples[chain, draw, last_time_step, home_idx]
            
            # Calculate expected goals (lambda)
            theta_home = beta_xG * np.log(xG_home + 0.01) + att_home - def_away + home_adv
            theta_away = beta_xG * np.log(xG_away + 0.01) + att_away - def_home
            
            lambda_home = np.exp(np.clip(theta_home, -5, 5))
            lambda_away = np.exp(np.clip(theta_away, -5, 5))
            
            # Sample from Poisson
            goals_home = np.random.poisson(lambda_home)
            goals_away = np.random.poisson(lambda_away)
            
            home_goals_samples.append(goals_home)
            away_goals_samples.append(goals_away)
    
    home_goals_samples = np.array(home_goals_samples)
    away_goals_samples = np.array(away_goals_samples)
    
    # Calculate scoreline probabilities
    scoreline_counts = defaultdict(int)
    for h, a in zip(home_goals_samples, away_goals_samples):
        scoreline_counts[(int(h), int(a))] += 1
    
    total_samples = len(home_goals_samples)
    scoreline_probs = {k: v/total_samples for k, v in scoreline_counts.items()}
    
    # Calculate outcome probabilities
    home_wins = np.sum(home_goals_samples > away_goals_samples) / total_samples
    draws = np.sum(home_goals_samples == away_goals_samples) / total_samples
    away_wins = np.sum(home_goals_samples < away_goals_samples) / total_samples
    
    # Expected goals
    exp_goals_home = np.mean(home_goals_samples)
    exp_goals_away = np.mean(away_goals_samples)
    
    # Common betting markets
    over_2_5 = np.sum((home_goals_samples + away_goals_samples) > 2.5) / total_samples
    over_1_5 = np.sum((home_goals_samples + away_goals_samples) > 1.5) / total_samples
    btts = np.sum((home_goals_samples > 0) & (away_goals_samples > 0)) / total_samples
    
    predictions.append({
        'home_team': home_team,
        'away_team': away_team,
        'scoreline_probs': scoreline_probs,
        'outcome_probs': {
            'home_win': home_wins,
            'draw': draws,
            'away_win': away_wins
        },
        'expected_goals': {
            'home': exp_goals_home,
            'away': exp_goals_away
        },
        'betting_markets': {
            'over_2.5': over_2_5,
            'over_1.5': over_1_5,
            'btts': btts
        }
    })
    
    print(f"{home_team} vs {away_team}")
    print(f"  Expected Goals: {exp_goals_home:.2f} - {exp_goals_away:.2f}")
    print(f"  Win Probabilities: Home {home_wins:.1%} | Draw {draws:.1%} | Away {away_wins:.1%}")
    print(f"  Over 2.5 Goals: {over_2_5:.1%} | BTTS: {btts:.1%}")
    print()

print("=== PREDICTIONS COMPLETE ===")
print(f"Generated probability distributions for {len(predictions)} matches")

=== GENERATING PREDICTIONS ===



NameError: name 'next_gameweek_fixtures' is not defined

In [ ]:
# Step 5: Display top scoreline probabilities for each match
print("=== MOST LIKELY SCORELINES ===\n")

for pred in predictions:
    print(f"{pred['home_team']} vs {pred['away_team']}")
    
    # Sort scorelines by probability
    sorted_scorelines = sorted(pred['scoreline_probs'].items(), 
                              key=lambda x: x[1], reverse=True)
    
    # Show top 10 most likely scorelines
    print("  Top 10 Scorelines:")
    for i, ((h, a), prob) in enumerate(sorted_scorelines[:10], 1):
        print(f"    {i}. {h}-{a}: {prob:.1%}")
    print()

In [ ]:
# Step 6: Compare model predictions to bookmaker odds (VALUE BETTING)
# This identifies where your model disagrees with the market

# Example bookmaker odds (decimal format)
# IMPORTANT: Replace with actual odds from bookmakers
bookmaker_odds = {
    'Arsenal vs Chelsea': {'home': 2.10, 'draw': 3.40, 'away': 3.50},
    'Liverpool vs Manchester United': {'home': 1.50, 'draw': 4.20, 'away': 6.50},
    'Manchester City vs Tottenham': {'home': 1.40, 'draw': 5.00, 'away': 7.00},
}

print("=== VALUE BETTING ANALYSIS ===\n")
print("Comparing model probabilities vs bookmaker-implied probabilities\n")

value_bets = []

for pred in predictions:
    match_name = f"{pred['home_team']} vs {pred['away_team']}"
    
    if match_name not in bookmaker_odds:
        print(f"⚠️  No odds available for {match_name}")
        continue
    
    odds = bookmaker_odds[match_name]
    
    # Convert odds to implied probabilities (removing bookmaker margin)
    implied_home = 1 / odds['home']
    implied_draw = 1 / odds['draw']
    implied_away = 1 / odds['away']
    
    # Bookmaker margin (overround)
    margin = implied_home + implied_draw + implied_away - 1.0
    
    # Fair probabilities (removing margin)
    fair_home = implied_home / (1 + margin)
    fair_draw = implied_draw / (1 + margin)
    fair_away = implied_away / (1 + margin)
    
    # Model probabilities
    model_home = pred['outcome_probs']['home_win']
    model_draw = pred['outcome_probs']['draw']
    model_away = pred['outcome_probs']['away_win']
    
    print(f"{match_name}")
    print(f"  Bookmaker margin: {margin:.1%}")
    print()
    print(f"  {'Outcome':<12} {'Model %':<10} {'Fair %':<10} {'Edge':<10} {'Odds':<8} {'Value'}")
    print(f"  {'-'*70}")
    
    # Check each outcome for value
    outcomes = [
        ('Home Win', model_home, fair_home, odds['home']),
        ('Draw', model_draw, fair_draw, odds['draw']),
        ('Away Win', model_away, fair_away, odds['away'])
    ]
    
    for outcome_name, model_prob, fair_prob, odd in outcomes:
        edge = model_prob - fair_prob
        expected_value = (model_prob * odd) - 1.0  # Expected profit per £1 bet
        
        # Flag as value bet if model probability significantly exceeds fair odds
        is_value = edge > 0.05 and expected_value > 0.10  # >5% edge and >10% EV
        
        value_indicator = "✓ VALUE BET" if is_value else ""
        
        print(f"  {outcome_name:<12} {model_prob:>8.1%} {fair_prob:>10.1%} "
              f"{edge:>9.1%} {odd:>7.2f} {value_indicator}")
        
        if is_value:
            value_bets.append({
                'match': match_name,
                'outcome': outcome_name,
                'model_prob': model_prob,
                'fair_prob': fair_prob,
                'odds': odd,
                'edge': edge,
                'expected_value': expected_value
            })
    
    print()

if value_bets:
    print(f"\n{'='*70}")
    print(f"FOUND {len(value_bets)} VALUE BET(S)")
    print(f"{'='*70}\n")
    
    for bet in value_bets:
        print(f"🎯 {bet['match']} - {bet['outcome']}")
        print(f"   Model: {bet['model_prob']:.1%} | Market: {bet['fair_prob']:.1%} | Edge: {bet['edge']:.1%}")
        print(f"   Odds: {bet['odds']:.2f} | Expected Value: {bet['expected_value']:+.1%}")
        print()
else:
    print("\n❌ No value bets found (model agrees with market)")
    print("   This is common - only bet when you have a genuine edge!")

In [ ]:
# Step 7: Visualize probability distributions
import matplotlib.pyplot as plt

# Pick first match to visualize
if predictions:
    pred = predictions[0]
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Outcome probabilities bar chart
    outcomes = ['Home Win', 'Draw', 'Away Win']
    probs = [pred['outcome_probs']['home_win'], 
             pred['outcome_probs']['draw'], 
             pred['outcome_probs']['away_win']]
    
    axes[0, 0].bar(outcomes, probs, color=['#2E7D32', '#FFA726', '#D32F2F'], alpha=0.7)
    axes[0, 0].set_ylabel('Probability', fontsize=12)
    axes[0, 0].set_title(f"{pred['home_team']} vs {pred['away_team']} - Match Outcome", 
                         fontsize=14, fontweight='bold')
    axes[0, 0].set_ylim(0, 1)
    for i, prob in enumerate(probs):
        axes[0, 0].text(i, prob + 0.02, f'{prob:.1%}', ha='center', fontsize=11)
    
    # 2. Top 15 scorelines
    sorted_scorelines = sorted(pred['scoreline_probs'].items(), 
                              key=lambda x: x[1], reverse=True)[:15]
    
    scoreline_labels = [f"{h}-{a}" for (h, a), _ in sorted_scorelines]
    scoreline_probs = [prob for _, prob in sorted_scorelines]
    
    axes[0, 1].barh(scoreline_labels, scoreline_probs, color='steelblue', alpha=0.7)
    axes[0, 1].set_xlabel('Probability', fontsize=12)
    axes[0, 1].set_title('Top 15 Most Likely Scorelines', fontsize=14, fontweight='bold')
    axes[0, 1].invert_yaxis()
    
    # 3. Goals distribution
    home_goals_dist = defaultdict(float)
    away_goals_dist = defaultdict(float)
    
    for (h, a), prob in pred['scoreline_probs'].items():
        home_goals_dist[h] += prob
        away_goals_dist[a] += prob
    
    max_goals = max(max(home_goals_dist.keys()), max(away_goals_dist.keys()))
    goals_range = list(range(max_goals + 1))
    
    home_probs = [home_goals_dist.get(g, 0) for g in goals_range]
    away_probs = [away_goals_dist.get(g, 0) for g in goals_range]
    
    x = np.arange(len(goals_range))
    width = 0.35
    
    axes[1, 0].bar(x - width/2, home_probs, width, label=pred['home_team'], 
                   color='#2E7D32', alpha=0.7)
    axes[1, 0].bar(x + width/2, away_probs, width, label=pred['away_team'], 
                   color='#D32F2F', alpha=0.7)
    axes[1, 0].set_xlabel('Goals', fontsize=12)
    axes[1, 0].set_ylabel('Probability', fontsize=12)
    axes[1, 0].set_title('Goals Distribution', fontsize=14, fontweight='bold')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(goals_range)
    axes[1, 0].legend()
    
    # 4. Betting markets
    markets = ['Over 1.5', 'Over 2.5', 'BTTS']
    market_probs = [
        pred['betting_markets']['over_1.5'],
        pred['betting_markets']['over_2.5'],
        pred['betting_markets']['btts']
    ]
    
    axes[1, 1].bar(markets, market_probs, color='#FF6F00', alpha=0.7)
    axes[1, 1].set_ylabel('Probability', fontsize=12)
    axes[1, 1].set_title('Popular Betting Markets', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylim(0, 1)
    for i, prob in enumerate(market_probs):
        axes[1, 1].text(i, prob + 0.02, f'{prob:.1%}', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n=== Match Summary: {pred['home_team']} vs {pred['away_team']} ===")
    print(f"Expected Score: {pred['expected_goals']['home']:.2f} - {pred['expected_goals']['away']:.2f}")
    print(f"Most Likely: {sorted_scorelines[0][0][0]}-{sorted_scorelines[0][0][1]} ({sorted_scorelines[0][1]:.1%})")
else:
    print("No predictions available to visualize")

In [ ]:
# Step 8: Export predictions to JSON for external use
import json

# Prepare data for export
export_data = {
    'generated_at': pd.Timestamp.now().isoformat(),
    'model_info': {
        'training_rounds': max_completed_round,
        'n_matches': len(input_data_latest.goals_home),
        'beta_xG': float(trace_latest.posterior['beta_xG'].mean().values)
    },
    'predictions': []
}

for pred in predictions:
    # Convert scoreline probs to serializable format
    scoreline_probs_list = [
        {'home_goals': int(h), 'away_goals': int(a), 'probability': float(prob)}
        for (h, a), prob in sorted(pred['scoreline_probs'].items(), 
                                   key=lambda x: x[1], reverse=True)[:20]  # Top 20
    ]
    
    export_data['predictions'].append({
        'home_team': pred['home_team'],
        'away_team': pred['away_team'],
        'outcome_probabilities': {
            'home_win': float(pred['outcome_probs']['home_win']),
            'draw': float(pred['outcome_probs']['draw']),
            'away_win': float(pred['outcome_probs']['away_win'])
        },
        'expected_goals': {
            'home': float(pred['expected_goals']['home']),
            'away': float(pred['expected_goals']['away'])
        },
        'betting_markets': {
            'over_2.5_goals': float(pred['betting_markets']['over_2.5']),
            'over_1.5_goals': float(pred['betting_markets']['over_1.5']),
            'btts': float(pred['betting_markets']['btts'])
        },
        'top_scorelines': scoreline_probs_list
    })

# Save to file
output_file = 'predictions_next_gameweek.json'
with open(output_file, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"✓ Predictions exported to {output_file}")
print(f"\nYou can now use this file to:")
print("  • Compare with live odds from betting APIs")
print("  • Track prediction accuracy over time")
print("  • Feed into automated betting systems")
print("  • Share predictions with others")